In [ ]:
# 多线程

Python3 线程中常用的两个模块为：

    - _thread
    - threading(推荐使用)

thread 模块已被废弃。用户可以使用 threading 模块代替。所以，在 Python3 中不能再使用"thread" 模块。为了兼容性，Python3 将 thread 重命名为 "_thread"。


## 开始学习Python线程

使用 threading 模块创建线程

我们可以通过直接从 threading.Thread 继承创建一个新的子类，并实例化后调用 start() 方法启动新线程，即它调用了线程的 run() 方法：

In [3]:
import time
import threading


def print_time(thread, counter, dalay):
    while counter:
        time.sleep(dalay)
        print("%d - %s - %s" % (thread.threadID, thread.name, time.time()))
        counter -= 1


class MyThread(threading.Thread):
    """a Thread class example.
    
    """
    def __init__(self, threadID, name, counter):
        super().__init__()
        self.threadID = threadID
        self.name = name
        self.counter = counter
    def run(self):
        print("Thread %s begin ", self.threadID)
        print_time(self, self.counter, 3)
        print("Thread %s end ", self.threadID)


# create thread
thread1 = MyThread(1, "Thread-A", 5)
thread2 = MyThread(2, "Thread-B", 6)

# run thread

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print('Exit main processes')

Thread %s begin Thread %s begin  1
 2
2 - Thread-B - 1628492085.73069741 - Thread-A - 1628492085.7211854

1 - Thread-A - 1628492088.73224142 - Thread-B - 1628492088.7322414

1 - Thread-A - 1628492091.73291782 - Thread-B - 1628492091.7329178

2 - Thread-B - 1628492094.7333051 - Thread-A - 1628492094.733305

1 - Thread-A - 1628492097.73438862 - Thread-B - 1628492097.7343886
Thread %s end  1

2 - Thread-B - 1628492100.735917
Thread %s end  2
Exit main processes


## 线程同步

如果多个线程共同对某个数据修改，则可能出现不可预料的结果，为了保证数据的正确性，需要对多个线程进行同步。

使用 Thread 对象的 Lock 和 Rlock 可以实现简单的线程同步，这两个对象都有 acquire 方法和 release 方法，对于那些需要每次只允许一个线程操作的数据，可以将其操作放到 acquire 和 release 方法之间。如下：

多线程的优势在于可以同时运行多个任务（至少感觉起来是这样）。但是当线程需要共享数据时，可能存在数据不同步的问题。

考虑这样一种情况：一个列表里所有元素都是0，线程"set"从后向前把所有元素改成1，而线程"print"负责从前往后读取列表并打印。

那么，可能线程"set"开始改的时候，线程"print"便来打印列表了，输出就成了一半0一半1，这就是数据的不同步。为了避免这种情况，引入了锁的概念。

锁有两种状态——锁定和未锁定。每当一个线程比如"set"要访问共享数据时，必须先获得锁定；如果已经有别的线程比如"print"获得锁定了，那么就让线程"set"暂停，也就是同步阻塞；等到线程"print"访问完毕，释放锁以后，再让线程"set"继续。

经过这样的处理，打印列表时要么全部输出0，要么全部输出1，不会再出现一半0一半1的尴尬场面。

In [7]:
import time
import threading

threadLock = threading.Lock()

def print_time(thread, counter, dalay):
    while counter:
        time.sleep(dalay)
        print("%d - %s - %s" % (thread.threadID, thread.name, time.time()))
        counter -= 1


class MyThread(threading.Thread):
    """a Thread class example.
    
    """
    def __init__(self, threadID, name, counter):
        super().__init__()
        self.threadID = threadID
        self.name = name
        self.counter = counter
    def run(self):
        print("Thread %s begin ", self.threadID)
        threadLock.acquire()
        print_time(self, self.counter, 3)
        threadLock.release()
        print("Thread %s end ", self.threadID)

threads = []

# create thread
thread1 = MyThread(1, "Thread-A", 5)
thread2 = MyThread(2, "Thread-B", 6)

# run thread
thread1.start()
thread2.start()

# 添加线程到线程列表
threads.append(thread1)
threads.append(thread2)

# 等待所有线程完成
for t in threads:
    t.join()

print('Exit main processes')

Thread %s begin  1
Thread %s begin  2
1 - Thread-A - 1628494495.9027846
1 - Thread-A - 1628494498.9034972
1 - Thread-A - 1628494501.904552
1 - Thread-A - 1628494504.9047923
1 - Thread-A - 1628494507.904993
Thread %s end  1
2 - Thread-B - 1628494510.9056232
2 - Thread-B - 1628494513.9070644
2 - Thread-B - 1628494516.9075193
2 - Thread-B - 1628494519.907722
2 - Thread-B - 1628494522.9085412
2 - Thread-B - 1628494525.909628
Thread %s end  2
Exit main processes


## 线程优先级队列（ Queue）

Python 的 Queue 模块中提供了同步的、线程安全的队列类，包括FIFO（先入先出)队列Queue，LIFO（后入先出）队列LifoQueue，和优先级队列 PriorityQueue。

这些队列都实现了锁原语，能够在多线程中直接使用，可以使用队列来实现线程间的同步。

Queue 模块中的常用方法:

Queue.qsize() 返回队列的大小
Queue.empty() 如果队列为空，返回True,反之False
Queue.full() 如果队列满了，返回True,反之False
Queue.full 与 maxsize 大小对应
Queue.get([block[, timeout]])获取队列，timeout等待时间
Queue.get_nowait() 相当Queue.get(False)
Queue.put(item) 写入队列，timeout等待时间
Queue.put_nowait(item) 相当Queue.put(item, False)
Queue.task_done() 在完成一项工作之后，Queue.task_done()函数向任务已经完成的队列发送一个信号
Queue.join() 实际上意味着等到队列为空，再执行别的操作

In [8]:
#!/usr/bin/python3

import queue
import threading
import time

exitFlag = 0

class myThread (threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
    def run(self):
        print ("开启线程：" + self.name)
        process_data(self.name, self.q)
        print ("退出线程：" + self.name)

def process_data(threadName, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            data = q.get()
            queueLock.release()
            print ("%s processing %s" % (threadName, data))
        else:
            queueLock.release()
        time.sleep(1)

threadList = ["Thread-1", "Thread-2", "Thread-3"]
nameList = ["One", "Two", "Three", "Four", "Five"]
queueLock = threading.Lock()
workQueue = queue.Queue(10)
threads = []
threadID = 1

# 创建新线程
for tName in threadList:
    thread = myThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

# 填充队列
queueLock.acquire()
for word in nameList:
    workQueue.put(word)
queueLock.release()

# 等待队列清空
while not workQueue.empty():
    pass

# 通知线程是时候退出
exitFlag = 1

# 等待所有线程完成
for t in threads:
    t.join()
print ("退出主线程")

开启线程：Thread-1
开启线程：Thread-2开启线程：Thread-3

Thread-2 processing One
Thread-1 processing TwoThread-3 processing Three

Thread-2 processing Four
Thread-3 processing Five
退出线程：Thread-2退出线程：Thread-1

退出线程：Thread-3
退出主线程
